### Thought process
First, count domains and create a table that has nrow corresponding to dom. count. As the string is very systematic, with a regional expression I can separate the domains with all their information given.  
The good thing here is that the sequence of domains is clear from the string.  
Most important is to know which domain is paired with which. As a human, when I find a bracket in the string, I look for its counterpart. This will have to be done in the second step and put the pairing info into a dictionary(?).

In [1]:
import numpy as np
import re

Angle: [-179, 180]

In [2]:
#error handling step 1
def check_string_valid(string):
    if re.search("[$ß€]", string): #all special characters?
        raise Exception("bad string")
        
check_string_valid("@90 a ß90 b")

Exception: bad string

In [3]:
string = '@90 a( B + ) @45 e* f1'
re.findall('@?[0-9]* ?[a-z|A-Z|0-9|\*]+[ \(]?\+?|\)', string)

['@90 a(', ' B +', ')', '@45 e* ', 'f1']

table(int) - cols: angle before dom, length, dom type?, if there is a strand break after  
dictionary - pairedness  
list of list - name of domains, their preliminary type 

In [51]:
def comprehend_string(string):
    #separate domains/count
    pattern = re.compile('@?-?[0-9]* ?[a-z|A-Z|0-9|\*]+[ \(]? ?\+?|\) ?\+?')
    dom_raw_list = pattern.findall(string)
    dom_count = len(dom_raw_list)
        
    #create empty data structures     
    struct_info_array = np.zeros((dom_count, 4), dtype = 'int') #4 cols rn
    name_list1 = []
    
    #loop through the raw domains, extract  
    for dom_i in range(len(dom_raw_list)):
        dom = dom_raw_list[dom_i]
        dom = dom.strip()
        
        #fill table
        if re.search("\s", dom):
            dom_parts = re.split("\s", dom)
            
            for segm in dom_parts:
                if segm[0] is "@": #col0 angle
                    segm = segm.lstrip("@")
                    struct_info_array[dom_i, 0] = int(segm) #col0 to write to
                    struct_info_array[dom_i, 1] = np.sign(int(segm)) #col1 to write to
                    
                elif segm is "+": #col2 strand break
                    struct_info_array[dom_i, 2] = 1         #col2 to write to
                else:
                    name_list1.append(segm) #only keep the name part further
        else:
            name_list1.append(dom)
            
    print(name_list1)
        
    #pairedness dictionary
    paired_dict = {}
    
    for dom_i2 in range(len(dom_raw_list)):
        if re.search("\(", dom_raw_list[dom_i2]):
            bracket_count = 1
            for next_dom in range(dom_i2+1, len(dom_raw_list)):
                if re.search("\(", dom_raw_list[next_dom]):
                    bracket_count += 1
                elif re.search("\)", dom_raw_list[next_dom]):
                    bracket_count -= 1
                if bracket_count == 0:
                    paired_dict[dom_i2] = next_dom
                    break
    
    #name list of list / finding hairpin loops
    #paireds in dict, hairpin loop known, other are unpaired -> substructure from these?
    name_final_lol = []
    
    for dom_i3 in range(len(name_list1)):
        
        #paireds
        if re.search("\(", name_list1[dom_i3]):
            typ = "paired"
            name = name_list1[dom_i3].strip("(+ ")
        
        #correcting closing bracket name
        elif re.search("\)", name_list1[dom_i3]):
            typ = "paired"
            index_of_pair = list(paired_dict.keys())[list(paired_dict.values()).index(dom_i3)]
            if re.search("\*", name_list1[index_of_pair]):
                name = name_list1[index_of_pair].strip("\(").strip("\*")
            else:
                name = name_list1[index_of_pair].strip("\(") + "*"
        
        #check hairpinloop on next domain
        elif re.search("\(", name_list1[dom_i3-1]) and re.search("\)", name_list1[dom_i3+1]):
            print(name_list1[dom_i3])
            if re.search("\+", name_list1[dom_i3]): #hairpin unless strand break
                typ = "lin. unpaired"
                name = name_list1[dom_i3].strip(" +")
            else: 
                typ = "hairpin"
                name = name_list1[dom_i3]
                
        #all other are unpaireds
        else:
            typ = "unpaired"
            name = name_list1[dom_i3].strip("+ ")
        
        name_final_lol.append([name, typ])
        
    return struct_info_array, paired_dict, name_final_lol

In [52]:
string = "@-45 d1S T2 @-135 b( a( t2( + ) ) c*( t1* + ) )" #bug with sb after t1*
comprehend_string(string)

['d1S', 'T2', 'b(', 'a(', 't2(', ')', ')', 'c*(', 't1*', ')', ')']
t1*


(array([[ -45,   -1,    0,    0],
        [   0,    0,    0,    0],
        [-135,   -1,    0,    0],
        [   0,    0,    0,    0],
        [   0,    0,    1,    0],
        [   0,    0,    0,    0],
        [   0,    0,    0,    0],
        [   0,    0,    0,    0],
        [   0,    0,    1,    0],
        [   0,    0,    0,    0],
        [   0,    0,    0,    0]]),
 {2: 10, 3: 6, 4: 5, 7: 9},
 [['d1S', 'unpaired'],
  ['T2', 'unpaired'],
  ['b', 'paired'],
  ['a', 'paired'],
  ['t2', 'paired'],
  ['t2*', 'paired'],
  ['a*', 'paired'],
  ['c*', 'paired'],
  ['t1*', 'hairpin'],
  ['c', 'paired'],
  ['b*', 'paired']])

In [6]:
string2 = "t1( c( a( + t2* ) b*( d2 t3 + ) ) )" #here is a bug with strand break within "hairpin"
comprehend_string(string2)

(array([[0, 0, 0],
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]),
 {0: 10, 1: 9, 2: 4, 5: 8},
 [['t1', 'paired'],
  ['c', 'paired'],
  ['a', 'paired'],
  ['t2*', 'hairpin'],
  ['a*', 'paired'],
  ['b*', 'paired'],
  ['d2', 'unpaired'],
  ['t3', 'unpaired'],
  ['b', 'paired'],
  ['c*', 'paired'],
  ['t1*', 'paired']])

In [7]:
string3 = "@-90 aa( @-135 i1 @-90 da( dl ) @-135 i2 @-90 aca( acl ) @-135 vr @-135 pa( pl ) @-135 i3 ) @0 c"
comprehend_string(string3)

(array([[ -90,    0,    0],
        [-135,    0,    0],
        [ -90,    0,    0],
        [   0,    0,    0],
        [   0,    0,    0],
        [-135,    0,    0],
        [ -90,    0,    0],
        [   0,    0,    0],
        [   0,    0,    0],
        [-135,    0,    0],
        [-135,    0,    0],
        [   0,    0,    0],
        [   0,    0,    0],
        [-135,    0,    0],
        [   0,    0,    0],
        [   0,    0,    0]]),
 {0: 14, 2: 4, 6: 8, 10: 12},
 [['aa', 'paired'],
  ['i1', 'unpaired'],
  ['da', 'paired'],
  ['dl', 'hairpin'],
  ['da*', 'paired'],
  ['i2', 'unpaired'],
  ['aca', 'paired'],
  ['acl', 'hairpin'],
  ['aca*', 'paired'],
  ['vr', 'unpaired'],
  ['pa', 'paired'],
  ['pl', 'hairpin'],
  ['pa*', 'paired'],
  ['i3', 'unpaired'],
  ['aa*', 'paired'],
  ['c', 'unpaired']])

determine_structure -  
1. Figure out multiloops  
2. Given angles determine the side for the substructure they are in. Using this, the whole column can be filled around the given info, if not provided, default is negative (-1) side.
3. complete angle/length col  
4. calculate  coords

In [33]:
def determine_structure(array, dictionary, list_of_lists):
    dom_count = len(list_of_lists)
    
    #1.
    
    #2. complete side col ~ get same side for paireds/hairpins, default to -1
    curr_side = -1
    for dom in range(dom_count):

        if dom in dictionary:
            if array[dom, 1] != 0:
                segside = array[dom, 1]
            elif array[dictionary[dom], 1] != 0:
                segside = array[dictionary[dom], 1]
            else:
                segside = curr_side
            
            #if hairpin
            if dom == dictionary[dom]-2:
                array[dom, 1] = segside
                array[dom+1, 1] = segside
                array[dom+2, 1] = segside
            
            #if not hairpin
            else:
                array[dom, 1] = segside
                array[dictionary[dom], 1] = segside
            
        else:
            if array[dom, 1] == 0:
                array[dom, 1] = curr_side
        
        curr_side = array[dom, 1] #update curr side after changes
        
    #3.
    
    #4.
        
    return array, list_of_lists

In [34]:
array, dictionary, list_of_lists = comprehend_string(string)
determine_structure(array, dictionary, list_of_lists)

(array([[ -45,   -1,    0,    0],
        [   0,   -1,    0,    0],
        [-135,   -1,    0,    0],
        [   0,   -1,    0,    0],
        [   0,   -1,    1,    0],
        [   0,   -1,    0,    0],
        [   0,   -1,    0,    0],
        [   0,   -1,    0,    0],
        [   0,   -1,    1,    0],
        [   0,   -1,    0,    0],
        [   0,   -1,    0,    0]]),
 [['d1S', 'unpaired'],
  ['T2', 'unpaired'],
  ['b', 'paired'],
  ['a', 'paired'],
  ['t2', 'paired'],
  ['t2*', 'paired'],
  ['a*', 'paired'],
  ['c*', 'paired'],
  ['t1*', 'hairpin'],
  ['c', 'paired'],
  ['b*', 'paired']])

In [35]:
str4 = "@-90 a( b ) @-90 c @90 d( e ) @90 f"
array, dictionary, list_of_lists = comprehend_string(str4)
determine_structure(array, dictionary, list_of_lists)

(array([[-90,  -1,   0,   0],
        [  0,  -1,   0,   0],
        [  0,  -1,   0,   0],
        [-90,  -1,   0,   0],
        [ 90,   1,   0,   0],
        [  0,   1,   0,   0],
        [  0,   1,   0,   0],
        [ 90,   1,   0,   0]]),
 [['a', 'paired'],
  ['b', 'hairpin'],
  ['a*', 'paired'],
  ['c', 'unpaired'],
  ['d', 'paired'],
  ['e', 'hairpin'],
  ['d*', 'paired'],
  ['f', 'unpaired']])